superclass
mro
mixin
https://realpython.com/inheritance-composition-python/

a parent class inherits from a parent class, like with genetics.

- child class == derived class == subclass
- parent class == base class == superclass


child inherits from parent. 
derived inherits from base.
sub inherits from super.

So if you have class X and Y, then when X(Y), X is the subclass and Y is the superclass.




## super method

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width
    
    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length=length, width=length) # you can just do: (length, length)

square = Square(4)
square.area()

16

In [ ]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    # no need to override init because it does the same as in Square
    def surface_area(self):
        face_area = super().area() # here we reuse area() from Rectangle
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length

cube = Cube(3)

display(cube.surface_area())
display(cube.volume())

54

27

super().area() is defined in Triangle and in Square. And here you can see it uses the one form Triangle, the first argument. That's why it crashes. This is because of the **method resolution order**.

In [ ]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height
    
    def area(self):
        return 0.5 * self.base * self.height

class RightPyramid(Triangle, Square):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
    
    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

pyramid = RightPyramid(2,4)
pyramid.area()

AttributeError: 'RightPyramid' object has no attribute 'height'

turns out there is a dunder to check out the method resolution order:

In [ ]:
RightPyramid.__mro__

(__main__.RightPyramid,
 __main__.Triangle,
 __main__.Square,
 __main__.Rectangle,
 object)

Now let's try to fix this by swapping the arguments.

In [ ]:
class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
        super().__init__(self.base) # call Square __init__
    
    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

display(RightPyramid.__mro__)
pyramid = RightPyramid(2,4)
display(pyramid.area())

(__main__.RightPyramid,
 __main__.Square,
 __main__.Rectangle,
 __main__.Triangle,
 object)

20.0

In the init, the line `super().__init__(self.base)` was added, so it can call square. Is there a way to choose which superclass you can call? The tutorial does not elaborate. It tells to use diferent method names or different method parameters.

In [ ]:
class Rectangle:
    def __init__(self, length, width, **kwargs):
        self.length = length
        self.width = width
        super().__init__(**kwargs) # why here?
    
    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width


class Square(Rectangle):
    def __init__(self, length, **kwargs):
        super().__init__(length=length, width=length, **kwargs)


class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length


class Triangle:
    def __init__(self, base, height, **kwargs):
        self.base = base
        self.height = height
        super().__init__(**kwargs) # why here?
    
    def tri_area(self):
        return 0.5 * self.base * self.height


class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height, **kwargs):
        self.base = base
        self.slant_height = slant_height
        kwargs["height"] = slant_height
        kwargs["length"] = base
        super().__init__(base = self.base, **kwargs)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

    
    def area_2(self):
        base_area = super().area()
        triangle_area = super().tri_area()
        return triangle_area * 4 + base_area

pyramid = RightPyramid(2,4)
display(pyramid.area())
display(pyramid.area_2())


20.0

20.0